In [1]:
import os
import pandas as pd
from tqdm import tqdm
from fasttext import FastText
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Запустить, расскомментировать, запустить ещё раз (с speller и без)

In [3]:
titles = pd.read_csv('./titles_speller.tsv', sep='\t', header=None)
titles.columns=['Id', 'Title']

# titles = pd.read_csv('./titles_speller.tsv', sep='\t', header=None)
# titles.columns=['Id', 'Title']

In [4]:
titles

,Id,Title
0,334761,ответы mail ru кто родился 4 июля\n
1,56349,наращивание ногтей акрилом описание достоинств...
2,551990,как узнать есть ли что то в собственности zaza...
3,224905,relp криминалистическое исследование речевого ...
4,389288,николай козлов синтон психология тренинги розе...
...,...,...
582162,458053,голод сергей львович москвин голод книга подго...
582163,474067,как сделать фотографию снимок с экрана монитор...
582164,179069,селективная парфюмерия нишевая парфюмерия стат...
582165,539701,10 установленных плагинов wordpress в блоге но...


In [5]:
queries = pd.read_csv('./query_speller.tsv', sep='\t', header=None)
queries.columns = ['Id', 'Query']

# queries = pd.read_csv('./query_speller.tsv', sep='\t', header=None)
# queries.columns = ['Id', 'Query']

In [6]:
queries

,Id,Query
0,0,13 причин почему
1,1,1 положительный и 1 отрицательный могут ли
2,2,2016 действует ли зао рождественская мануфактура
3,3,1 месяц после операции на кишечнике диета что ...
4,4,2 правды 1 ложь что можно придумать
...,...,...
6306,6306,является ли тойота харриер внедорожником
6307,6307,як можно очистить крейду
6308,6308,являются ли реактивы медицинскими изделиями
6309,6309,являются ли словообразовательными парами слова...


In [7]:
queries['Id'] = queries['Id'].apply(lambda x: int(x))

In [8]:
titles['Title'] = titles['Title'].astype('str').apply(lambda x: x.strip())
titles['Id'] = titles['Id'].apply(lambda x: int(x))
titles = titles.sort_values(by=['Id'])

In [9]:
sample_df = pd.read_csv('./Data/sample.csv', sep=',')
sample_df.columns = ['query_id', 'url_id']

marks_df = pd.read_csv('./Data/train.marks.tsv', sep='\t', header=None)
marks_df.columns = ['query_id', 'url_id', 'mark']
marks_df = marks_df.drop(columns=['mark'])

sample_df = sample_df.append(marks_df)

sample_df['Fasttext'] = None
sample_df['Fasttext_DP'] = None

In [10]:
corpus = titles['Title']

In [11]:
model_base = FastText.load_model('cc.ru.300.bin')
model_DP = FastText.load_model('ft_native_300_ru_wiki_lenta_lower_case.bin')

In [12]:
all_q = list(set(sample_df['query_id']))

In [13]:
for query in tqdm(all_q):
    docs_id = list(sample_df[sample_df['query_id'] == query]['url_id'])
    
    q = queries[queries['Id'] == query]['Query'][query]

    q_vec_base = model_base.get_sentence_vector(q)
    q_vec_DP = model_DP.get_sentence_vector(q)
    
    docs_base = []
    docs_DP = []
    for doc in docs_id:      
        docs_base.append(model_base.get_sentence_vector(corpus[doc]))
        docs_DP.append(model_DP.get_sentence_vector(corpus[doc]))
        
    fasttext = cosine_similarity([q_vec_base], docs_base)[0]
    fasttext_DP = cosine_similarity([q_vec_DP], docs_DP)[0]

    sample_df.loc[sample_df['query_id'] == query, 'Fasttext'] = fasttext
    sample_df.loc[sample_df['query_id'] == query, 'Fasttext_DP'] = fasttext_DP

100%|██████████| 6311/6311 [04:57<00:00, 21.23it/s]


In [14]:
sample_df.to_csv('Fasttext/FT_Spelled.tsv',sep='\t',index=False)

In [14]:
#sample_df.to_csv('Fasttext/FT_Spelled_Stemmed.tsv',sep='\t',index=False)